In [1]:
require('rstan')
require('tidyverse')


Loading required package: rstan
Loading required package: ggplot2
Loading required package: StanHeaders
rstan (Version 2.17.3, GitRev: 2e1f913d3ca3)
For execution on a local, multicore CPU with excess RAM we recommend calling
options(mc.cores = parallel::detectCores()).
To avoid recompilation of unchanged Stan programs, we recommend calling
rstan_options(auto_write = TRUE)
Loading required package: tidyverse
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ tibble  1.4.2     ✔ purrr   0.2.4
✔ tidyr   0.8.1     ✔ dplyr   0.7.4
✔ readr   1.1.1     ✔ stringr 1.3.0
✔ tibble  1.4.2     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks rstan::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()


In [2]:
require('data.table')

Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [14]:
# read in a file
options(stringsAsFactors=FALSE)
GWAS.DIR <- "../gwas1015"
DATA.DIR <- "../data/"
trait <- "Testosterone"
sex <- "zerosex"
gwas.out <- fread(sprintf("%s/ukb24983_v2_hg19.%s_%s.genotyped.glm.linear", GWAS.DIR, trait, sex), data.table=FALSE)

In [5]:
head(gwas.out)
nrow(gwas.out)

#CHROM POS    ID          REF ALT A1 TEST OBS_CT BETA        SE        
1 XY      60425 rs34557243  C   A   A  ADD  142309  0.00701977 0.01030930
2 XY      60454 rs28494123  A   G   G  ADD  141664  0.00598013 0.00368371
3 XY      61067 rs28590175  A   G   G  ADD  142636  0.00599071 0.01020780
4 XY      62079 rs117654552 C   G   G  ADD  142488  0.01012510 0.01021310
5 XY      62615 rs28491545  A   C   C  ADD  142421  0.00660512 0.00369493
6 XY     167755 rs73174453  G   T   T  ADD  142434 -0.00627778 0.00375782
  T_STAT    P        
1  0.680916 0.4959250
2  1.623400 0.1045060
3  0.586877 0.5572870
4  0.991387 0.3214980
5  1.787620 0.0738401
6 -1.670590 0.0948046

[1] 804470

### include X in vars to keep


In [28]:

vars.to.keep <- read.table(sprintf("%s/snp_filt_list2.txt" , DATA.DIR), header=FALSE)
head(vars.to.keep)

# read in all the chr qc
cX_ld <- read.table(sprintf("%s/chr_qc/chrX_ld.prune.in", DATA.DIR), header=FALSE)
cXY_ld <- read.table(sprintf("%s/chr_qc/chrXY_ld.prune.in", DATA.DIR), header=FALSE)
cX_tab <- read.table(sprintf("%s/chr_qc/chrX_qc_table.txt", DATA.DIR), sep=" ", header=TRUE)
cXY_tab <- read.table(sprintf("%s/chr_qc/chrXY_qc_table.txt", DATA.DIR), sep=" ", header=TRUE)
cXY_hardy <- read_tsv(sprintf("%s/chr_qc/hardy_XY.out.hardy", DATA.DIR))
cX_hardy <- read_tsv(sprintf("%s/chr_qc/hardy_X.out.hardy.x", DATA.DIR))

V1         
1 rs11240779 
2 rs59771807 
3 rs116452738
4 rs4970382  
5 rs13303101 
6 rs192998324

Parsed with column specification:
cols(
  `#CHROM` = col_character(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  HOM_REF_CT = col_integer(),
  HET_REF_CT = col_integer(),
  NONREF_CT = col_integer(),
  `O(HET_REF)` = col_double(),
  `E(HET_REF)` = col_double(),
  P = col_double()
)
Parsed with column specification:
cols(
  `#CHROM` = col_character(),
  ID = col_character(),
  REF = col_character(),
  ALT = col_character(),
  FEMALE_HOM_REF_CT = col_integer(),
  FEMALE_HET_REF_CT = col_integer(),
  FEMALE_NONREF_CT = col_integer(),
  MALE_REF_CT = col_integer(),
  MALE_ALT_CT = col_integer(),
  `O(FEMALE_HET_REF)` = col_double(),
  `E(FEMALE_HET_REF)` = col_double(),
  FEMALE_REF_FREQ = col_double(),
  MALE_REF_FREQ = col_double(),
  P = col_double()
)


In [21]:
table(cX_tab$keep==1)
table(cXY_tab$keep==1)

head(cX_tab)
head(cXY_tab)

filter(cX_tab, SNP %in% cX_ld$V1) %>% filter(LD !=1) %>% nrow()
filter(cXY_tab, SNP %in% cXY_ld$V1) %>% filter(LD !=1) %>% nrow()


filter(cX_tab, SNP %in% cX_ld$V1) %>% filter(LD ==1) %>% nrow()
filter(cXY_tab, SNP %in% cXY_ld$V1) %>% filter(LD==1) %>% nrow()


combinedX <- cX_tab %>% mutate(LD2=ifelse(SNP %in% cX_ld$V1,1,0))
combinedXY <- cXY_tab %>% mutate(LD2=ifelse(SNP %in% cXY_ld$V1,1,0))



FALSE  TRUE 
13045  5812 


FALSE  TRUE 
  829   528 

SNP           CHR A1 A2 MAF      F_MISS      LD keep
1 Affx-34464730 X   A  G  0.821453 0.000683898 0  0   
2 Affx-34470541 X   T  G  0.998817 0.001566410 1  1   
3 Affx-34480971 X   A  C  0.987194 0.010207300 1  1   
4 Affx-34492707 X   T  C  0.988038 0.002023030 1  1   
5 Affx-34492709 X   G  A  0.224857 0.000788325 0  0   
6 Affx-34494619 X   C  G  0.767836 0.001820320 0  0

SNP           CHR A1 A2 MAF      F_MISS      LD keep
1 Affx-34637689 XY  T  C  0.701461 0.003255680 0  0   
2 Affx-34642957 XY  T  G  0.925713 0.000702326 0  0   
3 Affx-34660664 XY  A  T  0.978064 0.002043500 0  0   
4 Affx-34691882 XY  T  C  0.418119 0.001973880 0  0   
5 Affx-34729416 XY  T  C  0.896423 0.835355000 0  0   
6 Affx-34734836 XY  G  A  0.189222 0.067742700 0  0

[1] 5789

[1] 438

[1] 4543

[1] 413

In [24]:
combinedX2 <- combinedX %>% select(-LD, -keep) %>% rename(LD=LD2) 
combinedXY2 <- combinedXY %>% select(-LD, -keep) %>% rename(LD=LD2) 
head(combinedX2)
head(combinedXY2)

SNP           CHR A1 A2 MAF      F_MISS      LD keep
1 Affx-34464730 X   A  G  0.821453 0.000683898 0  0   
2 Affx-34470541 X   T  G  0.998817 0.001566410 0  0   
3 Affx-34480971 X   A  C  0.987194 0.010207300 1  1   
4 Affx-34492707 X   T  C  0.988038 0.002023030 1  1   
5 Affx-34492709 X   G  A  0.224857 0.000788325 0  0   
6 Affx-34494619 X   C  G  0.767836 0.001820320 0  0

SNP           CHR A1 A2 MAF      F_MISS      LD keep
1 Affx-34637689 XY  T  C  0.701461 0.003255680 1  1   
2 Affx-34642957 XY  T  G  0.925713 0.000702326 1  1   
3 Affx-34660664 XY  A  T  0.978064 0.002043500 1  1   
4 Affx-34691882 XY  T  C  0.418119 0.001973880 0  0   
5 Affx-34729416 XY  T  C  0.896423 0.835355000 0  0   
6 Affx-34734836 XY  G  A  0.189222 0.067742700 0  0

In [30]:
head(cX_hardy)
head(cXY_hardy)

#CHROM ID            REF ALT FEMALE_HOM_REF_CT FEMALE_HET_REF_CT
1 X      rs60075487    A   G     7340             72122           
2 X      rs2306736     C   T    75908            130843           
3 X      Affx-92044070 AT  A        0                20           
4 X      rs5939320     G   A    26848            114413           
5 X      rs146462965   T   C        5              6222           
6 X      rs1970797     T   C   198341             60873           
  FEMALE_NONREF_CT MALE_REF_CT MALE_ALT_CT O(FEMALE_HET_REF) E(FEMALE_HET_REF)
1 185023            36077      186736      2.72688e-01       2.74337e-01      
2  56870           118635      103318      4.96330e-01       4.97392e-01      
3 237148                0      196223      8.43284e-05       8.43249e-05      
4 122715            70724      151757      4.33422e-01       4.34055e-01      
5 258426             2581      220674      2.35100e-02       2.32706e-02      
6   4852           192439       30333      2.30522e-01       2.31554e-01      
  FEMALE_REF_FREQ MALE_REF_FREQ P          
1 1.64096e-01     0.1619160     5.58886e-04
2 5.36109e-01     0.5345050     2.44207e-01
3 4.21642e-05     0.0000000     2.17114e-03
4 3.18417e-01     0.3178880     6.81506e-01
5 1.17739e-02     0.0115608     4.90229e-10
6 8.66365e-01     0.8638380     1.00389e-03

#CHROM ID          REF ALT HOM_REF_CT HET_REF_CT NONREF_CT O(HET_REF)
1 XY     rs34557243  A   C     1526      34508     450367    0.0709456 
2 XY     rs28494123  G   A   136003     230107     117847    0.4754700 
3 XY     rs28590175  G   A     4117      37497     446013    0.0768969 
4 XY     rs117654552 G   C     1714      35077     450347    0.0720063 
5 XY     rs28491545  C   A   136946     233527     116337    0.4797090 
6 XY     rs73174453  T   G   141590     240549     104614    0.4941910 
  E(HET_REF) P           
1 0.0742388  1.15550e-163
2 0.4992960  1.04200e-241
3 0.0893851   0.00000e+00
4 0.0759194  3.85634e-215
5 0.4991040  6.63162e-162
6 0.4971150   4.07944e-05

In [43]:
cXY_h <- left_join(combinedXY2, cXY_hardy %>% select(ID, P), by=c("SNP"="ID")) %>% rename(HWE_P=P)
head(cXY_h)

cX_h <- left_join(combinedX2, cX_hardy %>% select(ID, P), by=c("SNP"="ID")) %>% rename(HWE_P=P)
head(cX_h)


SNP           CHR A1 A2 MAF      F_MISS      LD keep HWE_P       
1 Affx-34637689 XY  T  C  0.701461 0.003255680 1  1     1.18165e-04
2 Affx-34642957 XY  T  G  0.925713 0.000702326 1  1    2.50053e-132
3 Affx-34660664 XY  A  T  0.978064 0.002043500 1  1     2.54957e-09
4 Affx-34691882 XY  T  C  0.418119 0.001973880 0  0     1.24136e-03
5 Affx-34729416 XY  T  C  0.896423 0.835355000 0  0     7.61234e-01
6 Affx-34734836 XY  G  A  0.189222 0.067742700 0  0     2.23538e-01

SNP           CHR A1 A2 MAF      F_MISS      LD keep HWE_P      
1 Affx-34464730 X   A  G  0.821453 0.000683898 0  0    0.00000e+00
2 Affx-34470541 X   T  G  0.998817 0.001566410 0  0    8.42647e-11
3 Affx-34480971 X   A  C  0.987194 0.010207300 1  1    1.51841e-01
4 Affx-34492707 X   T  C  0.988038 0.002023030 1  1    1.50668e-01
5 Affx-34492709 X   G  A  0.224857 0.000788325 0  0    1.75749e-07
6 Affx-34494619 X   C  G  0.767836 0.001820320 0  0    1.90858e-68

In [49]:
cX_keep <- cX_h  %>% filter(LD == 1) %>% filter(MAF >= 0.01) %>% filter(F_MISS <=0.1) %>% filter(HWE_P <= (10**(-7)))
cXY_keep <- cXY_h  %>% filter(LD == 1) %>% filter(MAF >= 0.01) %>% filter(F_MISS <=0.1) %>% filter(HWE_P <= (10**(-7)))


In [51]:
nrow(cX_keep)
nrow(cXY_keep)

head(cX_keep)

[1] 2450

[1] 279

In [59]:
vars.to.keep.all <- c(vars.to.keep$V1, cX_keep$SNP, cXY_keep$SNP)
write_tsv(data.frame(vars.to.keep.all), sprintf("%s/snp_filt_list2_wX.txt", DATA.DIR), col_names=FALSE)

In [60]:
length(vars.to.keep.all)

[1] 228780

In [62]:
length(intersect(vars.to.keep.all, gwas.out$ID))

[1] 228780

### make sure read in and speed up works 

In [66]:
source("../mixture_model_scripts/model_utils.R")


snps.to.keep <- read.table(sprintf("%s/snp_filt_list2_wX.txt", DATA.FOLDER), header=FALSE, 
    colClasses="character")

QUANT.SE.CUTOFF <- 0.2

loadDat <- function(trait){
    require('data.table')

    # for each trait
    list.ds <- lapply(list.prefixes, function(prefix) {
        dat.1 <- fread(sprintf("%s/ukb24983_v2_hg19.%s_%s.genotyped.glm.linear", GWAS.DIR, trait, prefix), data.table=FALSE)
        
        colnames(dat.1)[1:3] <- c("CHR", "BP", "SNP");
        
        rownames(dat.1) <- dat.1$SNP

        # remove NAs
        dat.2 <- dat.1[!is.na(dat.1$SE),]

        # SE filter
        dat.3 <- dat.2[dat.2$SE < QUANT.SE.CUTOFF,]

        # MAF filter
        dat.4 <- dat.3[dat.3$SNP %in% snps.to.keep$V1,]
        return(dat.4)

    })

    list.ds2 <- extractOverlappingRows(list.ds)
    # maybe keep the pvals? we seem to want these
    stan.obj <- extractDataStanMulti(list.ds2)
    return(stan.obj)
}

list.prefixes <- c("zerosex", "onesex")
dat <- loadDat("Testosterone")
str(dat, 2)

[1] 2
[1] 186175     12
List of 3
 $ dat:List of 5
  ..$ N : int 185340
  ..$ M : int 2
  ..$ B : num [1:185340, 1:2] 0.00702 0.00598 -0.000633 0.000558 0.005264 ...
  ..$ SE: num [1:185340, 1:2] 1.06e-04 1.36e-05 1.51e-05 7.44e-05 2.59e-04 ...
  ..$ K : num 2
 $ snp: Named chr [1:185340] "rs34557243" "rs28494123" "rs55771472" "rs6644964" ...
  ..- attr(*, "names")= chr [1:185340] "rs34557243" "rs28494123" "rs55771472" "rs6644964" ...
 $ chr: Named chr [1:185340] "XY" "XY" "XY" "XY" ...
  ..- attr(*, "names")= chr [1:185340] "XY" "XY" "XY" "XY" ...


In [ ]:
DATA.OUT <- sprintf("%s/1015", DATA.DIR)
dat$dat$K <- 2
    save(dat, file=sprintf("%s/dat_%s.RData", DATA.OUT, trait))
    fit1 <- stan(file = "../mixture_model_scripts/models/model1_no_loglik.stan",  
    data = dat$dat,    
    chains = 4, warmup = 100, iter = 100, cores = 4, refresh = 100)

[1] "LEARNING PARAMS"


### GENERATED QUANTITIES